## 사고데이터 + 격자 병합하는 전처리과정

In [1]:
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from datetime import datetime
import numpy as np

In [4]:
# area = gpd.GeoDataFrame.from_file('/Users/dangchan/Desktop/Github/IOT_hackathon/data/LSMD_ADM_SECT_UMD_11.shp', encoding='cp949') # 경로 수정
grid = gpd.GeoDataFrame.from_file('/Users/dangchan/Desktop/Github/IOT_hackathon/Seoul_Grid/grid.shp') # 경로 수정

In [6]:
# datetime series for each row(grid)
grid['dt'] = 0
dt_range = pd.date_range(start="2021-01-01", end="2022-01-01", freq='H')[:-1] # from 21/01/01 to 21/12/31 by hour
for i in tqdm(range(len(grid))):
    grid.dt[i] = dt_range

  0%|          | 0/2311 [00:00<?, ?it/s]/var/folders/6r/ndnspf4n6n7_m8rb9bs7qdvr0000gp/T/ipykernel_19222/3780216747.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grid.dt[i] = dt_range
100%|██████████| 2311/2311 [00:00<00:00, 103651.10it/s]


In [7]:
# Explode! (about 30s)
'''
Explode to grid(spatial) * temporal dataset(N=20244360) > about 460 MB
각 격자에 속한 시계열을 펼치는 과정
'''
grid_new = grid.explode('dt')
grid_new = grid_new.iloc[:,[1,2]]

In [10]:
# Merge with accident data
df_acc = gpd.read_file('accidentdata.gpkg')
grid_new['accident'] = 0

for i in tqdm(range(len(df_acc))):
    idx = np.where(grid.geometry.contains(df_acc.geometry[i]))[0][0]
    dt = pd.to_datetime(df_acc.datetime[i])
    grid_new.loc[(grid_new.index == idx) & (grid_new.dt == dt), 'accident'] = 1 # 각 사고의 인덱스와 일시에 대응하는 행 추출하여 accident(사고유무) 변수를 1로 취함

grid_new.loc[grid_new.accident == 1].head(5)
    

100%|██████████| 445/445 [00:25<00:00, 17.63it/s]


,geometry,dt,accident
51,"POLYGON ((938535.249 1951165.520, 938535.249 1...",2021-07-01 22:00:00,1
68,"POLYGON ((939035.249 1951665.520, 939035.249 1...",2021-05-06 21:00:00,1
89,"POLYGON ((939535.249 1952165.520, 939535.249 1...",2021-11-25 19:00:00,1
93,"POLYGON ((939535.249 1954165.520, 939535.249 1...",2021-01-27 21:00:00,1
97,"POLYGON ((940035.249 1943165.520, 940035.249 1...",2021-02-08 15:00:00,1


In [11]:
# 각자 파일로 저장
grid_new.to_pickle('accident_in_grid.pkl')